In [7]:
import pandas as pd

df = pd.read_csv('merged_result.csv', delimiter=';')

#filter
df = df[df['Role'] == 'User']
df = df[['Role', 'Message', 'Classification']]

In [8]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

# Inicializar o tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Codificar os rótulos em inteiros
label_encoder = LabelEncoder()
df['encoded_classification'] = label_encoder.fit_transform(df['Classification'])

# Exibir as primeiras linhas para verificar
print(df.head())


/Users/rodrigo/dev/uc3m/uc3m/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/rodrigo/dev/uc3m/uc3m/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/rodr

    Role                                            Message  \
0   User  que funcion hace el metodo stack push y stack pop   
3   User              como saber la altura de un arbol dado   
6   User  como saber la altura de un arbol dado unos val...   
9   User                                   que es una deque   
12  User                     que es un NullPointerException   

          Classification  encoded_classification  
0   Conceptual Questions                       2  
3   Conceptual Questions                       2  
6   Conceptual Questions                       2  
9   Conceptual Questions                       2  
12  Conceptual Questions                       2  


/Users/rodrigo/dev/uc3m/uc3m/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Exibir o tamanho dos conjuntos
print(f'Tamanho do conjunto de treino: {len(train_df)}')
print(f'Tamanho do conjunto de teste: {len(test_df)}')


Tamanho do conjunto de treino: 219
Tamanho do conjunto de teste: 55


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenização
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Defina alguns hiperparâmetros
MAX_LEN = 128
BATCH_SIZE = 16

# Criar os datasets
train_dataset = TextClassificationDataset(
    texts=train_df['Message'].to_numpy(),
    labels=train_df['encoded_classification'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

test_dataset = TextClassificationDataset(
    texts=test_df['Message'].to_numpy(),
    labels=test_df['encoded_classification'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

# Criar os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [11]:
from transformers import BertModel

class TextClassifier(torch.nn.Module):
    def __init__(self, n_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = torch.nn.Dropout(p=0.3)
        self.out = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        return self.out(output)

# Instanciar o modelo
model = TextClassifier(n_classes=len(label_encoder.classes_))


## Treinamento

In [22]:
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Movendo o modelo para a GPU se disponível
model = model.to(device)

# Definir otimizador e função de perda
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss().to(device)

def train_epoch(model, data_loader, loss_fn, optimizer, device, n_examples):
    model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, torch.tensor(losses).mean()

# Treinamento por épocas
EPOCHS = 1

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        len(train_dataset)
    )

    print(f'Train loss {train_loss:.4f}, accuracy {train_acc:.4f}')

Epoch 1/1
----------


100%|██████████| 14/14 [01:55<00:00,  8.24s/it]

Train loss 0.5114, accuracy 0.8767


## Teste de modelo

In [23]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, torch.tensor(losses).mean()

# Avaliar o modelo no conjunto de teste
test_acc, test_loss = eval_model(
    model,
    test_loader,
    loss_fn,
    device,
    len(test_dataset)
)

print(f'Test loss {test_loss:.4f}, accuracy {test_acc:.4f}')


100%|██████████| 4/4 [00:07<00:00,  1.85s/it]

Test loss 1.6510, accuracy 0.5455


## Salvar o modelo

In [24]:
torch.save(model.state_dict(), 'interaction_model.bin')

## Carregar o modelo

In [25]:
import torch
from transformers import BertTokenizer

# Carregar o modelo salvo
model = TextClassifier(n_classes=len(label_encoder.classes_))
model.load_state_dict(torch.load('interaction_model.bin'))
model = model.to(device)
model.eval()

# Inicializar o tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/Users/rodrigo/dev/uc3m/uc3m/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
def predict_class(model, tokenizer, sentence, max_len=128):
    # Tokenizar a frase
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Colocar o modelo em modo de avaliação e fazer a previsão
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs, dim=1)
    
    # Converter a previsão numérica de volta para o nome da classe original
    predicted_class_name = label_encoder.inverse_transform([predicted_class.cpu().item()])[0]

    return predicted_class_name


In [55]:
# Exemplo de frase
frase = "que tipos de datos no pueden usar los operadores logicos"

# Fazer a previsão
classe_prevista = predict_class(model, tokenizer, frase)

# Exibir o resultado
print(f'Classification is: {classe_prevista}')


Classification is: Conceptual Questions
